In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py_trees
import logging
from deep_traffic.libtraffic import env
from behavior_tree.agent import BasicBehaviorTreeAgent, GeneticProgrammingBehaviorTreeAgent
from behavior_tree.condition_checker import BasicConditionChecker, GeneticProgrammingConditionChecker
from behavior_tree.tree import BasicBehaviorTree, GeneticProgrammingBehaviorTree, ConditionSequenceNode, IntersectConditionSequenceNode
from train import train_GP_BT
from runner import play_episode
from config import *

In [3]:
logging.basicConfig(level=logging.DEBUG)

##### Environment

In [4]:
e = env.DeepTraffic(lanes_side=LANES_SIDE, 
                    patches_ahead=PATCHES_AHEAD,
                    patches_behind=PATCHES_BEHIND, 
                    history=HISTORY, 
                    obs=OBS)
obs_shape = e.obs_shape
print(obs_shape)

(7, 5, 20)


In [5]:
state = e.reset()
print(state.shape)

(7, 5, 20)


In [6]:
print(state[0].T)

[[1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     0.0325 1.     1.     1.    ]
 [1.     0.0325 1.     1.     1.    ]
 [1.     0.0325 0.0325 0.0325 1.    ]
 [1.     0.0325 0.0325 0.0325 1.    ]
 [1.     1.     0.0325 0.0325 1.    ]
 [1.     1.     0.0325 0.0325 1.    ]
 [1.     1.     1.     1.     1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     0.0325 0.016  1.    ]
 [1.     1.     0.0325 1.     1.    ]
 [0.0325 1.     0.0325 1.     1.    ]
 [0.0325 1.     0.0325 1.     1.    ]
 [0.0325 1.     1.     1.     1.    ]]


##### ConditionChecker

In [7]:
condition_checker = GeneticProgrammingConditionChecker()
condition_checker.update_state(state[0])

In [8]:
print(condition_checker.state.shape)

(5, 20)


In [9]:
print(condition_checker.state.T)

[[1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     1.     1.     1.     0.016 ]
 [1.     0.0325 1.     1.     1.    ]
 [1.     0.0325 1.     1.     1.    ]
 [1.     0.0325 0.0325 0.0325 1.    ]
 [1.     0.0325 0.0325 0.0325 1.    ]
 [1.     1.     0.0325 0.0325 1.    ]
 [1.     1.     0.0325 0.0325 1.    ]
 [1.     1.     1.     1.     1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     1.     0.016  1.    ]
 [1.     1.     0.0325 0.016  1.    ]
 [1.     1.     0.0325 1.     1.    ]
 [0.0325 1.     0.0325 1.     1.    ]
 [0.0325 1.     0.0325 1.     1.    ]
 [0.0325 1.     1.     1.     1.    ]]


##### BTAgent

In [10]:
agent = GeneticProgrammingBehaviorTreeAgent()

In [11]:
# display tree
agent.display_tree()

[o] root
    [-] InitRoot
        [o] SafetyNode
            --> SafetyCond
            --> SafetyAct
        --> LearnAct



In [12]:
# tick
agent.update_condition_checker(state[0])
agent.update_blackboard()
print(agent.blackboard)

Blackboard Client
  Client Data
    name              : root
    namespace         : /
    unique_identifier : a185cc02-9918-437a-8fc7-17b461221469
    read              : {'/action'}
    write             : {'/speed', '/cell_condition', '/enable_learning'}
    exclusive         : set()
  Variables
    /action          : -
    /speed           : 65.0
    /cell_condition  : {'0': False, '1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': False, '12': False, '13': False, '14': False, '15': False, '16': False, '17': True, '18': True, '19': True, '20': False, '21': False, '22': False, '23': False, '24': False, '25': True, '26': True, '27': True, '28': True, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': True, '48': True, '49':

##### Training

In [13]:
%%time
#py_trees.logging.level = py_trees.logging.Level.DEBUG
train_GP_BT(e, agent, episodes=8, steps=200)

INFO:root:Episode 1/32
INFO:root:step 1/200: Learn Action triggered
C:\Users\shiro\Anaconda3\envs\GA-BT\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\shiro\Anaconda3\envs\GA-BT\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
INFO:root:step 1/200: new action sets, appended as a new Behavior
INFO:root:step 3/200: Learn Action triggered
INFO:root:step 3/200: duplicated action sets, intersect conditions with an existed Behavior
INFO:root:step 4/200: Learn Action triggered
INFO:root:step 4/200: duplicated action sets, intersect conditions with an existed Behavior
INFO:root:step 5/200: Learn Action triggered
INFO:root:step 5/200: duplicated action sets, intersect conditions with an existed Behavior
INFO:root:step 6/200: Learn Action triggered
INFO:root:step 6/200: duplicated action sets, intersect conditions with an existed Beha

Wall time: 1h 5min 29s


In [14]:
# display tree
agent.display_tree()

[o] root
    [-] Behavior5
        [-] Cond915
            --> F0
            --> F1
            --> F2
            --> T3
            --> T4
            --> T5
            --> T6
            --> F7
            --> F8
            --> F9
            --> F10
            --> F11
            --> F12
            --> F13
            --> F14
            --> F15
            --> F16
            --> F17
            --> F18
            --> F19
            --> T20
            --> F21
            --> F22
            --> F23
            --> F24
            --> F25
            --> F26
            --> F27
            --> F28
            --> F29
            --> F30
            --> F31
            --> F32
            --> F33
            --> F34
            --> F35
            --> F36
            --> F37
            --> F38
            --> F39
            --> F40
            --> F41
            --> F42
            --> F43
            --> F44
            --> F45
            --> F46
            --> T47
   

##### Play Episode

In [1]:
agent.blackboard.enable_learning = False
play_episode(e, agent, verbose=True, reset=True)

NameError: name 'agent' is not defined